In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.3.2-bin-hadoop2.7')
import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logr_project').getOrCreate()

In [2]:
data = spark.read.csv('../Python-and-Spark-for-Big-Data/Spark_for_Machine_Learning/Logistic_Regression/customer_churn.csv', 
                      inferSchema=True, header=True)

In [3]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [4]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [5]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
assembler = VectorAssembler(inputCols=[
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
    ], outputCol='features')

In [8]:
output = assembler.transform(data)

In [20]:
final_data = output.select('features', 'churn')

In [21]:
train, test = final_data.randomSplit([.7, .3])

In [22]:
from pyspark.ml.classification import LogisticRegression

In [23]:
logr_churn = LogisticRegression(labelCol='churn')

In [24]:
fitted_model = logr_churn.fit(train)

In [25]:
train_summary = fitted_model.summary

In [26]:
train_summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                612|                612|
|   mean|0.17320261437908496|0.13725490196078433|
| stddev| 0.3787318897504401| 0.3443977362185023|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [28]:
pred_and_labels = fitted_model.evaluate(test)

In [29]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[27.0,8628.8,1.0,...|    0|[5.15147424230666...|[0.99424247970332...|       0.0|
|[28.0,8670.98,0.0...|    0|[7.86727022611108...|[0.99961706803874...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.02728269873257...|[0.73638875126795...|       0.0|
|[28.0,11128.95,1....|    0|[3.72518789360561...|[0.97645897040837...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.31546791904197...|[0.91015000909193...|       0.0|
|[29.0,11274.46,1....|    0|[4.05183282661973...|[0.98290678733824...|       0.0|
|[29.0,12711.15,0....|    0|[5.25236509662540...|[0.99479214159896...|       0.0|
|[29.0,13240.01,1....|    0|[6.33557267208455...|[0.99823100419359...|       0.0|
|[29.0,13255.05,1....|    0|[3.62757463750323...|[0.97410765944901...|       0.0|
|[30.0,8403.78,1

In [30]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='churn')

In [31]:
auc = evaluator.evaluate(pred_and_labels.predictions)

In [32]:
auc

0.754284649776453

In [33]:
final_model = logr_churn.fit(final_data)

In [34]:
new_customers = spark.read.csv('../Python-and-Spark-for-Big-Data/Spark_for_Machine_Learning/Logistic_Regression/new_customers.csv', 
                               inferSchema=True, header=True)

In [35]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [36]:
test_new_customers = assembler.transform(new_customers)

In [37]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [40]:
final_results = final_model.transform(test_new_customers)

In [43]:
final_results.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [44]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                null|            null|
|    min|Andrew Mccall|              22.0|            100.0|          